In [ ]:
"""
Author: Annam.ai IIT Ropar
Team Name: TechTales
Team Members: Mohita, Siya Kumar
Leaderboard Rank: <140>
"""

# Import libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from zipfile import ZipFile
from google.colab import files

# Step 1: Upload and extract training data
print("Please upload your soil_type.zip file")
uploaded = files.upload()
zip_filename = next(iter(uploaded))

# Extract data
extract_dir = '/content/soil_data'
os.makedirs(extract_dir, exist_ok=True)
with ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print("Extraction complete!")

# Find the directory containing our classes
train_dir = None
for root, dirs, files in os.walk(extract_dir):
    if all(folder in dirs for folder in ['Alluvial', 'Black', 'Clay', 'Red']):
        train_dir = root
        break

if train_dir is None:
    print("Error: Could not find directory with all 4 soil class folders")
    print("Actual directory structure:")
    !ls -R {extract_dir}
    raise ValueError("Training directory not found with all required class folders")

# Image parameters
img_width, img_height = 150, 150
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Verify number of classes
num_classes = len(train_generator.class_indices)
print(f"\nNumber of classes detected: {num_classes}")
print("Class indices:", train_generator.class_indices)

# Build model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=5),
        ModelCheckpoint('best_model.keras', save_best_only=True)
    ]
)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

# Save final model in modern Keras format
model.save('soil_classifier.keras')
print("Model saved as 'soil_classifier.keras'")

# Download the model
files.download('soil_classifier.keras')
